In [ ]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from skimage import color

In [ ]:
BASE_PATH = '/kaggle/input/landscape-pictures'
BATCH_SIZE = len(os.listdir(BASE_PATH))

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1/255.0
)

train_data = train_datagen.flow_from_directory(
    '/kaggle/input/',
    target_size = (224,224),
    batch_size = BATCH_SIZE
)

In [ ]:
X = [] #grayscale images
y = [] #ground truth for X (other 2 channels)
X_test = []

i = 0
for image, _ in train_data:
    
    if i > 4200:
        X_test.append(image)
    else: 
        image = color.rgb2lab(image)
        print(image.shape)
        X.append(image[:, :, 0].reshape(224,224,1))
        y.append(image[:, :, 1:]/128)
    i += 1

X = np.array(X)
y = np.array(y)
X_test = np.array(X_test)

In [ ]:
def build_encoder():
    encoder = keras.Sequential()
    encoder.add(layers.Input(shape=(224, 224, 1)))
    encoder.add(layers.Conv2D(64, (3,3), strides=2, padding='same', activation='relu'))
    encoder.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
    encoder.add(layers.Conv2D(128, (3,3), strides= 2, padding='same', activation='relu'))
    encoder.add(layers.Conv2D(256, (3,3), padding='same', activation='relu'))
    encoder.add(layers.Conv2D(256, (3,3), strides= 2, padding='same', activation='relu'))
    encoder.add(layers.Conv2D(512, (3,3), padding='same', activation='relu'))
    encoder.add(layers.Conv2D(512, (3,3), padding='same', activation='relu'))
    encoder.add(layers.Conv2D(1024, (3,3), padding='same', activation='relu'))
    

    # Decoder stage
    encoder.add(layers.Conv2D(512, (3,3), padding='same', activation='relu'))
    encoder.add(layers.Conv2D(256, (3,3), padding='same', activation='relu'))
    encoder.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
    encoder.add(layers.UpSampling2D(size=(2,2)))
    encoder.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    encoder.add(layers.Conv2D(32, (3,3), padding='same', activation='relu'))

    encoder.add(layers.UpSampling2D(size=(2,2)))
    encoder.add(layers.Conv2D(16, (3,3), padding='same', activation='relu'))
    encoder.add(layers.Conv2D(2, (3,3), padding='same', activation='tanh'))
    encoder.add(layers.UpSampling2D(size=(2,2)))
    
    return encoder
model = build_encoder()
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001)

model.compile(optimizer='adam', metrics=['acc'], loss='mean_squared_error')
history = model.fit(X, y, epochs = 50, batch_size = BATCH_SIZE, steps_per_epoch = X.shape[0]//BATCH_SIZE, validation_split = 0.3, verbose = 1, callbacks=[reduce_lr])

In [ ]:
for i in range(50, 100):
    
    test_img = color.rgb2lab(X_test[i])
    plt.figure(figsize=(8, 6), dpi=80)
    plt.subplot(1,3,1)
    gray = np.zeros( (224, 224, 1))
    gray[:,:,0] = test_img[:, :, 0]
    plt.title("Graysacle Image")
    plt.imshow(gray, cmap='gray')
    
    plt.subplot(1,3,2)
    pred = np.zeros((224, 224, 3))
    pred[:,:,0] = test_img[:, :, 0]
    ab = model.predict(test_img[:, :, 0].reshape((1,224,224,1))) 
    ab = ab*128
    pred[:,:,1:] = ab
    pred = color.lab2rgb(pred)
    plt.title("Encoder Output")
    plt.imshow(pred)
    
    plt.subplot(1,3,3)
    plt.title("Ground Truth")
    plt.imshow(color.lab2rgb(test_img))
    plt.show()

In [ ]:
model.summary()